# **Práctica 4: Codificación Q-aria con ruido**

Realizada por Adrián Pérez García el 22/03/2021

Se importan las librerias necesarias

In [1]:
import math
import numpy as np
import itertools as it

### Funciones auxiliares

Función encargada de obtener un texto de un fichero

In [2]:
def getText(path):
    # Se obtiene el texto del fichero
    file = open(path, 'r', encoding='utf8')
    text = file.read()
    file.close()
    
    return text

Función encargada de obtener la lista binaria a decodificar de un fichero

In [3]:
def getData(path):
    # Se utiliza la funcion ya creada para leer el fichero y se formatea como codigo quitando las ,
    return getText(path).replace(",","").replace("\n", "")

Función encargada de obtener la matriz de un fichero

In [4]:
def getRawMatrix(path):
    # Se lee el texto utilizando la funcion ya creada
    text = getText(path)
    # Se obtienen filas y columnas (Solo valido para el formato dado en los ficheros de datos)
    rows = len(text.split("\n"))
    i = 0
    columns = 1
    while(text[i] != "]"):
        if(text[i] == ","):
            columns += 1
        i += 1
    # Se formatea la matriz quitando todo elemento no numerico
    return text.replace("\n","").replace("[", "").replace("]", "").replace(",","").replace(" ", ""), rows, int(columns)

Función encargada de convertir una matriz en formato String a Numpy

In [5]:
def getNumpyMatrix(matrixRaw, rows, columns):
    matrix = np.zeros((rows, columns), dtype=object)
    for i in range(rows):
        for j in range(columns):
            matrix[i][j] = int(matrixRaw[i*columns+j])
    return matrix

Funciones auxiliares para el calculo de sindromes (Obtener palabra como matriz y calculo de sindromes)

In [6]:
def getWordAsTransponseMatrix(word):
    matrix = np.zeros((len(word),1), dtype=object)
    for i in range(len(word)):
        matrix[i][0] = int(word[i])
    return matrix

def calculateSyndrome(word, matrixH, mod):
    # Se obtiene el sindrome como producto matricial de la matriz de control por la palabra
    syndrome = np.dot(matrixH , getWordAsTransponseMatrix(word)).transpose()
    # Se convierte de matriz con valores en N a lista con valores en su modulo
    syndromeList = []
    for j in range(len(syndrome[0])):
        syndromeList.append(syndrome[0][j] % mod)
        
    return syndromeList

Funciones auxiliares para el calculo de todas las palabras de peso 2 (Calculo de palabras y calculo de números combinatorios)

In [7]:
def combinatorial(n, k):
    # Se calcula recursivamente n sobre k
    if n >= 0 and k == 0:
        return 1
    if n == 0 and k > 0:
        return 0
    return combinatorial(n - 1, k - 1) + combinatorial(n - 1, k) 
            
def getWeight(error):
    weight = 0
    for element in error:
        if(element != 0):
            weight += 1
    return weight

def generateErrorsPatternBoard(wordLength, weight, mod):
        
    board = np.zeros((1, wordLength))
    boardIter = 1
    
    allErrors = list(it.product(range(mod), repeat=wordLength))
        
    for i in range(1, weight+1):
        for error in allErrors:
            if(getWeight(error) == i):
                board = np.concatenate((board, np.array(error)[np.newaxis]),0)
    return board
            


Función auxiliar para la resta binaria (Convierte el número a lista para poder restarla)

In [8]:
def getNumberAsList(number):
    numberList = []
    numberStr = str(number)
    for element in numberStr:
        numberList.append(int(element))
    return numberList

def listSubstraction(list1, list2, mod):
    resultList = []
    for i in range(len(list1)):
        resta = (int(list1[i]) - int(list2[i])) % mod
        resultList.append(resta)
    return resultList
        

Función decodificadora (Igual que en la practica 3)

In [13]:
def decodeMessage(code, alphabet, rowsA, columnsA, mod):
    # Se realiza la decodificacion lineal
    linearDecoded = ""
    for i in range(0, len(code), columnsA):
        linearDecoded += code[i:i+rowsA]
        if(len(code[i:i+rowsA]) != rowsA):
            print("Hay cola (" +code[i:i+rowsA]+ ")\n")
    
    # Se realiza la decodificacion con la fuente
    message = ""
    minLength = math.ceil(math.log(len(alphabet),mod))
    print("Longitud mínima en bloque: " + str(minLength) + "\n")
    for i in range(0, len(linearDecoded), minLength):
        message += alphabet[int(linearDecoded[i:i+minLength], mod)]
    
    return message

Función correctora de ruido

In [14]:
def correctData(code, hammingDistance, matrixA, mod):
    # Se generan las variables necesarias para corregir el ruido 

    # G sera la composicion de I (tamaño de filas de A) concatenada con A 
    matrixG = np.concatenate((np.identity(len(matrixA)), matrixA), 1)
    print("Tamaño de la matriz generadora: " +str(len(matrixG))+ "x" +str(len(matrixG[0])))

    # H sera la composicion de -A transpuesta concatenada con la identidad de tamaño columnas de A
    matrixH = np.concatenate((matrixA.transpose(), np.identity(len(matrixA[0]))), 1)
    print("Tamaño de la matriz de control: " +str(len(matrixH))+ "x" +str(len(matrixH[0])))
    
    # Se calcula la capacidad correctora en base a la distancia de Hamming
    errors = math.floor((hammingDistance - 1) / 2)
    
    # Se separa el dato en palabras de longitud columnas de G y se convierte a un array numpy para poder trabajar con el
    words = []
    wordLength = len(matrixG[0])
    for i in range(0, len(code), wordLength):
        words.append(code[i:i+wordLength])
        if(len(code[i:i+wordLength]) != wordLength):
            print("Hay cola(" +code[i:i+wordLength] + ")")
    words = np.array(words, dtype=object)

    # Se muestra la informacion basica del codigo dividido
    if(len(words[-1]) != wordLength):
        validWords =  len(words) - 1
        print("Total de palabras (incluida la cola): " + str(len(words)))
    else:
        validWords =  len(words)
        print("Total de palabras (no hay cola): " + str(len(words)))
        
        
    # Se aplica el algoritmo del lider

    # Paso 1: Calcular el sindrome de cada palabra
    syndromes = []
    for i in range(validWords):
        # Se utiliza una funcion auxiliar para calcular los sindromes
        syndromes.append(calculateSyndrome(words[i], matrixH, mod))
        
    for i in range(len(syndromes)):
        print(words[i])
        print(syndromes[i])

    # Paso 2.1: Obtener el tablero de errores de patron y de sindromes incompleto
    errorsPatternBoard = generateErrorsPatternBoard(wordLength, errors, mod)
    incompleteSyndromesBoard = []
    for i in range(0, len(errorsPatternBoard)):
        incompleteSyndromesBoard.append(calculateSyndrome(errorsPatternBoard[i], matrixH, mod))
        
    print("\nTablero de sindromes incompleto")
    for i in range(len(incompleteSyndromesBoard)):
        print(incompleteSyndromesBoard[i])
        
    # Paso 2.2 y 3: Buscar el sindrome en el tablero y restar la palabra del sindrome con el error del sindrome del tablero
    for i in range(len(syndromes)):
        for j in range(len(incompleteSyndromesBoard)):
            if(syndromes[i] == incompleteSyndromesBoard[j]):
                words[i]= listSubstraction(getNumberAsList(words[i]), errorsPatternBoard[j], mod)
              
    # Se reagrupa el codigo sin ruido para devolverlo
    freeCode = ""
    for word in words:
        for digit in word:
            freeCode += str(digit)
    return freeCode

### Problema a resolver

**La práctica 4 es una continuación de la práctica 3. Los procesos descritos tanto para la codificación de la fuente como para la codificación lineal son los mismos que en la práctica anterior. Sin embargo, ahora se añade una dificultad adicional. En este caso se ha aplicado un simulador de ruido a la secuencia binaria que sale tras hacer la
codificación lineal.**

In [15]:
# Se obtienen el alfabeto, la matriz y el dato de los ficheros
alphabet = getText("./data/alphabet3.txt")
data = getData("./data/data3.txt")
rawA, rowsA, columnsA = getRawMatrix("./data/matrix3.txt")
matrixA = getNumpyMatrix(rawA, rowsA, columnsA)
print("Matriz A:\n" +str(matrixA)+ "\n")
print("Alfabeto: " +alphabet+ "\n")
print("Dato: " +data+ "\n")
mod = 5
hammingDistance = 5

# Se corrige el dato eliminando el ruido
correctedData = correctData(data, hammingDistance, matrixA, mod)
print("\nDato sin ruido: " +correctedData+ "\n")

# Se obtiene el mensaje igual que en la practica 3 (Se imprime tomando dos espacios como un cambio de linea)
print("Mensaje: " +decodeMessage(correctedData, alphabet, len(matrixA), len(matrixA) + len(matrixA[0]), mod).replace("  ", "\n"))

Matriz A:
[[1 2 3 4 1]
 [2 3 4 1 2]]

Alfabeto: ABCDEFGHIJKLM NÑOPQRSTUVWXYZ

Dato: 20142312112341021112120142312343214420433024413244131423134132443321434224132103214320142310143214212224242142314312341343331311000004224132200433041413241243214343331314142310143214143403312234122431423141423114142311

Tamaño de la matriz generadora: 2x7
Tamaño de la matriz de control: 5x7
Hay cola(1)
Total de palabras (incluida la cola): 32
2014231
[3.0, 3.0, 3.0, 1.0, 3.0]
2112341
[0.0, 4.0, 3.0, 3.0, 0.0]
0211121
[0.0, 2.0, 4.0, 4.0, 0.0]
2014231
[3.0, 3.0, 3.0, 1.0, 3.0]
2343214
[2.0, 1.0, 0.0, 2.0, 2.0]
4204330
[3.0, 3.0, 3.0, 1.0, 3.0]
2441324
[4.0, 2.0, 0.0, 4.0, 4.0]
4131423
[4.0, 2.0, 0.0, 4.0, 4.0]
1341324
[1.0, 2.0, 3.0, 4.0, 1.0]
4332143
[3.0, 4.0, 0.0, 3.0, 3.0]
4224132
[0.0, 3.0, 1.0, 1.0, 0.0]
1032143
[4.0, 4.0, 4.0, 3.0, 4.0]
2014231
[3.0, 3.0, 3.0, 1.0, 3.0]
0143214
[1.0, 1.0, 1.0, 2.0, 1.0]
2122242
[1.0, 4.0, 2.0, 3.0, 1.0]
4214231
[4.0, 3.0, 2.0, 1.0, 4.0]
4312341
[1.0, 4.0, 2.0, 3.0